In [2]:
import heapq
from collections import deque

graph = {
    "Arad": {"Zerind": 75, "Timisoara": 118, "Sibiu": 140},
    "Zerind": {"Arad": 75, "Oradea": 71},
    "Oradea": {"Zerind": 71, "Sibiu": 151},
    "Timisoara": {"Arad": 118, "Lugoj": 111},
    "Lugoj": {"Timisoara": 111, "Mehadia": 70},
    "Mehadia": {"Lugoj": 70, "Drobeta": 75},
    "Drobeta": {"Mehadia": 75, "Craiova": 120},
    "Craiova": {"Drobeta": 120, "Pitesti": 138, "Rimnicu Vilcea": 146},
    "Sibiu": {"Arad": 140, "Oradea": 151, "Fagaras": 99, "Rimnicu Vilcea": 80},
    "Rimnicu Vilcea": {"Sibiu": 80, "Craiova": 146, "Pitesti": 97},
    "Fagaras": {"Sibiu": 99, "Bucharest": 211},
    "Pitesti": {"Rimnicu Vilcea": 97, "Craiova": 138, "Bucharest": 101},
    "Bucharest": {"Fagaras": 211, "Pitesti": 101, "Urziceni": 85, "Giurgiu": 90},
    "Giurgiu": {"Bucharest": 90},
    "Urziceni": {"Bucharest": 85, "Hirsova": 98, "Vaslui": 142},
    "Hirsova": {"Urziceni": 98, "Eforie": 86},
    "Eforie": {"Hirsova": 86},
    "Vaslui": {"Urziceni": 142, "Iasi": 92},
    "Iasi": {"Vaslui": 92, "Neamt": 87},
    "Neamt": {"Iasi": 87}
}

heuristics = {
    "Arad": 366, "Bucharest": 0, "Craiova": 160, "Drobeta": 242, "Eforie": 161,
    "Fagaras": 176, "Giurgiu": 77, "Hirsova": 151, "Iasi": 226, "Lugoj": 244,
    "Mehadia": 241, "Neamt": 234, "Oradea": 380, "Pitesti": 100, "Rimnicu Vilcea": 193,
    "Sibiu": 253, "Timisoara": 329, "Urziceni": 80, "Vaslui": 199, "Zerind": 374
}

def bfs(start, goal):
    queue = deque([(start, [start], 0)])
    visited = set()
    while queue:
        node, path, cost = queue.popleft()
        if node == goal:
            return path, cost
        if node not in visited:
            visited.add(node)
            for neighbor, distance in graph[node].items():
                queue.append((neighbor, path + [neighbor], cost + distance))
    return None

def ucs(start, goal):
    pq = [(0, start, [start])]
    visited = set()
    while pq:
        cost, node, path = heapq.heappop(pq)
        if node == goal:
            return path, cost
        if node not in visited:
            visited.add(node)
            for neighbor, distance in graph[node].items():
                heapq.heappush(pq, (cost + distance, neighbor, path + [neighbor]))
    return None

def greedy_bfs(start, goal):
    pq = [(heuristics[start], start, [start])]
    visited = set()
    while pq:
        _, node, path = heapq.heappop(pq)
        if node == goal:
            return path, sum(graph[path[i]][path[i+1]] for i in range(len(path)-1))
        if node not in visited:
            visited.add(node)
            for neighbor in graph[node]:
                heapq.heappush(pq, (heuristics[neighbor], neighbor, path + [neighbor]))
    return None

def iddfs(start, goal, max_depth=10):
    def dls(node, path, cost, depth):
        if depth == 0:
            return None
        if node == goal:
            return path, cost
        for neighbor, distance in graph[node].items():
            if neighbor not in path:
                result = dls(neighbor, path + [neighbor], cost + distance, depth - 1)
                if result:
                    return result
        return None

    for depth in range(max_depth):
        result = dls(start, [start], 0, depth)
        if result:
            return result
    return None

def compare_algorithms(start, goal):
    results = {
        "BFS": bfs(start, goal),
        "UCS": ucs(start, goal),
        "Greedy Best-First Search": greedy_bfs(start, goal),
        "IDDFS": iddfs(start, goal)
    }
    results = {k: v for k, v in results.items() if v}
    results = sorted(results.items(), key=lambda x: x[1][1])

    print("Algorithm Comparison (Ascending Order of Path Cost):\n")
    for algo, (path, cost) in results:
        print(f"{algo}: Path = {' -> '.join(path)}, Cost = {cost}")

start, goal = "Timisoara", "Iasi"
compare_algorithms(start, goal)


Algorithm Comparison (Ascending Order of Path Cost):

UCS: Path = Timisoara -> Arad -> Sibiu -> Rimnicu Vilcea -> Pitesti -> Bucharest -> Urziceni -> Vaslui -> Iasi, Cost = 855
BFS: Path = Timisoara -> Arad -> Sibiu -> Fagaras -> Bucharest -> Urziceni -> Vaslui -> Iasi, Cost = 887
IDDFS: Path = Timisoara -> Arad -> Sibiu -> Fagaras -> Bucharest -> Urziceni -> Vaslui -> Iasi, Cost = 887
Greedy Best-First Search: Path = Timisoara -> Lugoj -> Mehadia -> Drobeta -> Craiova -> Pitesti -> Bucharest -> Urziceni -> Vaslui -> Iasi, Cost = 934
